In [1]:
import pandas as pd
full_df = pd.read_csv('test-v5.csv')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence=sentence.lower()
    sentence=re.sub('[^\w\s]',' ', sentence) #removes punctuations
    sentence=re.sub('_',' ', sentence) #removes punctuations
    sentence=re.sub('\d+',' ', sentence) #removes digits
    cleaned=' '.join([w for w in sentence.split() if not w in stop]) # removes english stopwords
    cleaned=' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    
#selecting only nouns and adjectives
    cleaned=' '.join([w for w in cleaned.split() if not len(w)<=2 ]) #removes single lettered words and digits
    cleaned=cleaned.strip()
    return cleaned
	  
full_df['cleaned'] = full_df['description'].apply(lambda x: cleaning_text(x))

In [2]:
full_df.head()

,aptID,bathrooms,bedrooms,building_id,created,description,display_address,latitude,longitude,listing_id,...,fireplace,roof_deck,outdoor_space,high_speed_internet,balcony,swimming_pool,garden_or_patio,wheelchair_access,common_outdoor_space,cleaned
0,0,1.0,1,79780be1514f645d7e6be99a3de696c5,2016-06-11 05:29:00,Large with awesome terrace--accessible via bed...,Suffolk Street,40.7185,-73.9865,7142618,...,0,0,1,0,0,0,0,0,0,large awesome terrace accessible bedroom livin...
1,1,1.0,2,NaN,2016-06-24 06:36:00,Prime Soho - between Bleecker and Houston - Ne...,Thompson Street,40.7278,-74.0000,7210040,...,0,0,0,0,0,0,0,0,0,prime soho bleecker houston stainless steel ma...
2,2,1.0,0,NaN,2016-06-17 01:23:00,Spacious studio in Prime Location. Cleanbuildi...,Sullivan Street,40.7260,-74.0026,7174566,...,0,0,0,0,0,0,0,0,0,spacious studio prime location management supe...
3,3,1.0,2,f9c826104b91d868e69bd25746448c0c,2016-06-21 05:06:00,For immediate access call Bryan.<br /><br />Bo...,Jones Street,40.7321,-74.0028,7191391,...,0,0,0,0,0,0,0,0,0,immediate access call bryan bond new york real...
4,5,1.0,1,81062936e12ee5fa6cd2b965698e17d5,2016-06-16 07:24:00,Beautiful TRUE 1 bedroom in a luxury building ...,Exchange Place,40.7054,-74.0095,7171695,...,0,1,0,1,0,0,0,1,0,beautiful true bedroom luxury financial distri...


In [3]:
full_df.count()

aptID                   74659
bathrooms               74659
bedrooms                74659
building_id             62281
created                 74659
description             69434
display_address         74434
latitude                74659
longitude               74659
listing_id              74659
manager_id              74659
price                   74659
street_address          74646
created.Day             74659
created.Month           74659
created.Year            74659
created.Date            74659
created.WDay            74659
created.WDayLbl         74659
created.Week            74659
created.Hour            74659
created.Yday            74659
created.MWeek           74659
photoCount              74659
featureCount            74659
dining_room             74659
pre_war                 74659
laundry_in_building     74659
dishwasher              74659
hardwood_floors         74659
dogs                    74659
cats                    74659
doorman                 74659
elevator  

In [5]:
trimDF = full_df[['aptID','cleaned']]
trimDF.head()

,aptID,cleaned
0,0,large awesome terrace accessible bedroom livin...
1,1,prime soho bleecker houston stainless steel ma...
2,2,spacious studio prime location management supe...
3,3,immediate access call bryan bond new york real...
4,5,beautiful true bedroom luxury financial distri...


In [6]:
trimDF.to_csv('trimTEST.csv')

In [7]:
csv_file = raw_input('Enter the name of your input file: ')
txt_file = raw_input('Enter the name of your output file: ')

text_list = []

with open(csv_file, "r") as my_input_file:
    for line in my_input_file:
        line = line.split(",", 2)
        text_list.append(" ".join(line))

with open(txt_file, "w") as my_output_file:
    my_output_file.write("#1\n")
    my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write("  " + line)
    print('File Successfully written.')

Enter the name of your input file: trimTEST.csv
Enter the name of your output file: trimTEST.txt
File Successfully written.


In [9]:
from gensim.models import doc2vec
import nltk
import re

def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("trimTEST.txt")):
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])

# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save('description_test.model')

In [10]:
len(model.docvecs)

74659

In [123]:
len(test)

49352

In [11]:
model = doc2vec.Doc2Vec.load('description_test.model')

In [31]:
print model.most_similar(positive=["bedroom"], negative=["small"], topn=3)

[('ultra', 0.4796402156352997), ('penthouse', 0.4264823794364929), ('gorgeous', 0.40383094549179077)]


In [12]:
test = pd.read_csv('trimTEST.csv')
test['docvecs'] = model.docvecs
test.head()

,Unnamed: 0,aptID,cleaned,docvecs
0,0,0,large awesome terrace accessible bedroom livin...,"[-0.0028886, 0.0437069, -0.0148692, 0.0542286,..."
1,1,1,prime soho bleecker houston stainless steel ma...,"[0.0101035, 0.0087176, 0.0126455, 0.00310056, ..."
2,2,2,spacious studio prime location management supe...,"[-0.0249806, 0.0270556, 0.00232058, 0.00207128..."
3,3,3,immediate access call bryan bond new york real...,"[-0.00745866, 0.0300075, -0.0172987, 0.0360009..."
4,4,5,beautiful true bedroom luxury financial distri...,"[0.00599064, -0.000611184, 0.022732, 0.0030536..."


In [13]:
test['sum_vec'] = test['docvecs'].apply(lambda v: sum(v))
test.head()

,Unnamed: 0,aptID,cleaned,docvecs,sum_vec
0,0,0,large awesome terrace accessible bedroom livin...,"[-0.0028886, 0.0437069, -0.0148692, 0.0542286,...",-0.223342
1,1,1,prime soho bleecker houston stainless steel ma...,"[0.0101035, 0.0087176, 0.0126455, 0.00310056, ...",0.206371
2,2,2,spacious studio prime location management supe...,"[-0.0249806, 0.0270556, 0.00232058, 0.00207128...",-0.215066
3,3,3,immediate access call bryan bond new york real...,"[-0.00745866, 0.0300075, -0.0172987, 0.0360009...",-0.171405
4,4,5,beautiful true bedroom luxury financial distri...,"[0.00599064, -0.000611184, 0.022732, 0.0030536...",-0.300729


In [14]:
description_vec = test[['aptID','sum_vec']]
description_vec.head()

,aptID,sum_vec
0,0,-0.223342
1,1,0.206371
2,2,-0.215066
3,3,-0.171405
4,5,-0.300729


In [15]:
len(description_vec)

74659

In [16]:
description_vec.to_csv('description_vec_test_v2.csv', index =False)